In [2]:
import requests
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd
from pprint import pprint
import time
import re

In [3]:
with open('hot_json.json',newline='') as hot:
    data = json.load(hot)
    df = pd.json_normalize(data)

In [5]:
df.to_csv('hot_first.csv', mode='w', encoding='utf-8-sig')

,SEQ,TSEQNO,SUBJECT,PHOTOPATH1,CCORLORNM,CITYNM,KM1,SALAMT1,NAME,CARYY,...,TEL,CCNUM_R,GOLDMEMTYPE,GOLDCODE,BRANDCNM,HOTCERCD,WARDUR,HOT_RECOMMEND,OUTFLG,SPRICE
0,1,236581,豐田ALTIS 1800cc 2009年 黑色0頭款可全貸,https://www.hotcar.com.tw/UPLOAD/CW/HF1002/260...,黑色系,高雄市,19.8,19.6萬,上暘汽車,2009,...,07-7313801-,1.8,金鑽車商,1,Toyota 豐田,Y,2,N,N,0
1,2,236578,三菱得利卡2400cc 2016年框式附加吊桿0頭款可全貸,https://www.hotcar.com.tw/UPLOAD/CW/HF1002/260...,深藍系,高雄市,8.2,35.8萬,上暘汽車,2016,...,07-7313801-,2.4,金鑽車商,1,Mitsubishi 三菱,Y,2,N,N,0
2,3,236571,鈴木吉利 1590cc 2013年銀色0頭款可全貸,https://www.hotcar.com.tw/UPLOAD/CW/HF1002/260...,銀色系,高雄市,5.3,28.6萬,上暘汽車,2013,...,07-7313801-,1.6,金鑽車商,1,Suzuki 鈴木,Y,2,N,N,0
3,4,236495,新手駕駛首選 顯眼 小台 好停車 好照顧,https://www.hotcar.com.tw/UPLOAD/CW/HD1076/260...,淺綠系,台中市,9.3,23.8萬,聖傑汽車(烏日店),2013,...,04-23361166-,1.5,金鑽車商,1,Nissan 日產,Y,2,N,N,0
4,5,236493,Aero 旗艦版 電動座椅調整 4安 多合一影音,https://www.hotcar.com.tw/UPLOAD/CW/HD1076/260...,白色系,台中市,4.3,37.8萬,聖傑汽車(烏日店),2015,...,04-23361166-,1.8,金鑽車商,1,Nissan 日產,Y,2,N,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,6266,187472,"車況優,無改裝,操控佳,省油",https://www.hotcar.com.tw/UPLOAD/CW/HE2002/198...,深藍系,嘉義市,16.9,28.8萬,龍順汽車,2012,...,05-2358933-,1.2,,99,Mitsubishi 三菱,,0,N,N,0
6266,6267,180860,忠泰汽車 實車實價 2007 MAZDA 5 七人座 天窗版,https://www.hotcar.com.tw/UPLOAD/CW/HC1089/194...,淺藍系,桃園市,10,29.8萬,忠泰汽車,2007,...,03-355-9933,2.0,,99,Mazda 馬自達,,0,N,N,0
6267,6268,175198,"少開,車況優,馬力強,操控佳,空間大",https://www.hotcar.com.tw/UPLOAD/CW/HD2031/184...,黑色系,彰化縣,14.7,46.8萬,竣富汽車,2004,...,04-7330088-,3.0,,99,BMW 寶馬,,0,N,N,0
6268,6269,173304,"家庭用車,馬力強,空間大,操控佳,車況優",https://www.hotcar.com.tw/UPLOAD/CW/HD2031/182...,銀色系,彰化縣,16.2,14.8萬,竣富汽車,2005,...,04-7330088-,2.4,,99,Mitsubishi 三菱,N,0,N,N,0


In [8]:
df['TSEQNO']

0       236581
1       236578
2       236571
3       236495
4       236493
         ...  
6265    187472
6266    180860
6267    175198
6268    173304
6269    156514
Name: TSEQNO, Length: 6270, dtype: int64

In [ ]:
url_test = 'https://www.hotcar.com.tw/app/main/CWA/CWA060.js'
headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
r=requests.get(url_test,headers=headers)
r.text

In [6]:
# for d in df['TSEQNO'][:1]:
#     url = f'https://www.hotcar.com.tw/CWA/CWA060.html?TSEQNO={d}'
# headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
#     dc = DesiredCapabilities.CHROME
#     dc[''] = { 'Table1':'Array' }
# url = 'https://www.hotcar.com.tw/CWA/CWA060.html?TSEQNO=200399'

df1=pd.DataFrame(columns=['id',''])

driver = webdriver.Chrome('../chromedriver')
driver.get(url)
driver.maximize_window()
time.sleep(3)
soup=BeautifulSoup(driver.page_source,'lxml')

wdtype=soup.find(class_='tablein').find_next_sibling().find(class_='text')
print(wdtype.text)
address=soup.find(class_='table web').find(class_='text')
print(address.text)
photo=soup.find('img',class_='rsImg rsMainSlideImage').get('src')
print(photo)
component=soup.find('div', id='components').find_all('li')
for i in component:
    if i.text.strip() in equipment_all1:
        equipments[i] = 1

driver.close()

二輪傳動
彰化縣溪湖鎮湖東里員鹿路2段168號(溪湖NISSAN汽車正對面)
https://www.hotcar.com.tw/UPLOAD/CW/HD2008/2140861.jpg


In [ ]:
columns = ['id','car_brand','car_model','car_body','car_year','car_mileage','car_price','car_model_year','car_color','car_door','car_seat',
           'car_cylinderVolumn','car_cylinderType','car_driveMode','car_fuel','car_gear','car_title','car_photo','dealer_id','dealer_address',
           'verified','ABS','SRS','TPMS','TCS','ESP','isofix','cruise_sys','ACC','LDW','BSM','keyless','sunroof','HID','LED','electric_seat',
           'auto_AC','IVI_sys','EPB','SW_button','alu_wheel','leather_seat','fog_lamp','parking_radar','parking_camera','electric_reardoor',
           'electric_slidingdoor','panorama_roof','shift_paddle','panorama_camera']


In [ ]:
['安全氣囊', 'ABS', '定速', '倒車影像', '倒車雷達', '衛星導航', '胎壓偵測', '霧燈', 'ESP', '車道偏移系統', '車側盲點偵測系統', '鋁圈', '電動後視鏡', 'HID頭燈', '天窗', '電動尾門', '電動滑門', '全景天窗', '皮椅', '恆溫空調', 'PUSH START', 'ISOFIX', 'CD', 'VCD', 'DVD', '液晶螢幕', '電動座椅', '方向盤換檔撥片']

In [15]:
import urllib.request as req

In [15]:
url_try='https://www.hotcar.com.tw/SSAPI45/API/SPRetB?Token=VfaU+LJXyYZp7Nr3mFhCQtBfZ/rL2AQmOjkOW4W1uZVumEKn0wIHcD/RsdkmgB8di2Y9HFgUS/7HFxHm4m9eACLvfBCTdBEGoGqcd6RDUeZNSwlOrVeFarS9bEalGyz6'
url = 'https://www.hotcar.com.tw/CWA/CWA060.html?TSEQNO=236581'

requestdata={"SPNM":"CWA060Q_2018","SVRNM":["HOTCARAPP"],"PARMS":["236581","https://www.hotcar.com.tw","https://www.hotcar.com.tw/image/nophoto.png","",""]}

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
          'Content-Type':'application/json;charset=UTF-8',
          'Cookie':'_gcl_au=1.1.939139616.1624256679; _fbp=fb.2.1624256679494.1805338918; _ga=GA1.3.915183504.1624256680; __BWfp=c1624256679548xd6b06476d; clientID=915183504.1624256680; ASP.NET_SessionId=nrxwl2lrgxcsoaq4zw4jlvlk; Token=VfaU+LJXyYZp7Nr3mFhCQtBfZ/rL2AQmOjkOW4W1uZVumEKn0wIHcD/RsdkmgB8di2Y9HFgUS/7HFxHm4m9eACLvfBCTdBEGoGqcd6RDUeZNSwlOrVeFarS9bEalGyz6; _gid=GA1.3.1032427582.1624859448; Tagtoo_pta=pta_03+_&lta+_; _TUCI_T=sessionNumber+18806&pageView+18806&Search+18806; _TUCS=1; _TUCI=sessionNumber+5197&ECId+387&hostname+www.hotcar.com.tw&pageView+21786&Search+7000',
          'connection':'keep-alive',
          'Referer':'https://www.hotcar.com.tw/CWA/CWA060.html?TSEQNO=236581'}

# request = req.requests(url_try,headers=headers,data=json.dumps(requestdata).encode('utf-8'))

r=requests.get(url,headers=headers)
r.encoding='utf-8'
pprint(r.text)

('<!DOCTYPE html>\r\n'
 '<html lang="zh-tw">\r\n'
 '<head>\r\n'
 '    <meta charset="utf-8">\r\n'
 '    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n'
 '    <meta name="viewport" content="width=device-width, initial-scale=1, '
 'maximum-scale=1.0, user-scalable=0">\r\n'
 '    <title>HOT好車大聯盟|優質二手車</title>\r\n'
 '    <meta name="fragment" content="!">\r\n'
 '    <meta property="og:site_name" content="HOT好車大聯盟|優質二手車">\r\n'
 '    <meta property="og:type" content="website">\r\n'
 '    <meta property="og:url" '
 'content="\'https://www.hotcar.com.tw/CWA/CWA060.html">\r\n'
 '    <meta property="og:image" content="">\r\n'
 '    <meta property="og:title" content="HOT好車大聯盟|優質二手車">\r\n'
 '    <meta property="og:keywords" '
 'content="HOT,HOT大聯盟,HOT中古車,HOT認證中古車,好車大聯盟,HOT好車大聯盟">\r\n'
 '    <meta property="og:description" content="" class="intro">\r\n'
 '    <meta property="og:description" '
 'content="HOT好車大聯盟推薦您二手車，讓我們與您一起挑選最高品質的認證車、保固車，讓您買到符合期待的優質好車！" >\r\n'
 '    <meta property="og:

In [ ]:
VfaU+LJXyYZp7Nr3mFhCQtBfZ/rL2AQmOjkOW4W1uZVumEKn0wIHcD/RsdkmgB8di2Y9HFgUS/7HFxHm4m9eACLvfBCTdBEGoGqcd6RDUeZNSwlOrVeFarS9bEalGyz6

In [ ]:
VfaU+LJXyYZp7Nr3mFhCQtBfZ/rL2AQmOjkOW4W1uZVumEKn0wIHcD/RsdkmgB8di2Y9HFgUS/7HFxHm4m9eACLvfBCTdBEGoGqcd6RDUeZNSwlOrVeFarS9bEalGyz6

In [ ]:
VfaU%2BLJXyYZp7Nr3mFhCQtBfZ%2FrL2AQmOjkOW4W1uZVumEKn0wIHcD%2FRsdkmgB8di2Y9HFgUS%2F7HFxHm4m9eACLvfBCTdBEGoGqcd6RDUeZNSwlOrVeFarS9bEalGyz6